## 1. Data preparation

In [ ]:
# https://figshare.com/articles/dataset/MACCROBAT2018/9764942
# https://brat.nlplab.org/standoff.html

In [1]:
!mkdir MACCROBAT2018

In [4]:
!unzip /content/MACCROBAT2018.zip -d ./MACCROBAT2018

Archive:  /content/MACCROBAT2018.zip
  inflating: ./MACCROBAT2018/15939911.ann  
  inflating: ./MACCROBAT2018/15939911.txt  
  inflating: ./MACCROBAT2018/16778410.ann  
  inflating: ./MACCROBAT2018/16778410.txt  
  inflating: ./MACCROBAT2018/17803823.ann  
  inflating: ./MACCROBAT2018/17803823.txt  
  inflating: ./MACCROBAT2018/18236639.ann  
  inflating: ./MACCROBAT2018/18236639.txt  
  inflating: ./MACCROBAT2018/18258107.ann  
  inflating: ./MACCROBAT2018/18258107.txt  
  inflating: ./MACCROBAT2018/18416479.ann  
  inflating: ./MACCROBAT2018/18416479.txt  
  inflating: ./MACCROBAT2018/18561524.ann  
  inflating: ./MACCROBAT2018/18561524.txt  
  inflating: ./MACCROBAT2018/18666334.ann  
  inflating: ./MACCROBAT2018/18666334.txt  
  inflating: ./MACCROBAT2018/18787726.ann  
  inflating: ./MACCROBAT2018/18787726.txt  
  inflating: ./MACCROBAT2018/18815636.ann  
  inflating: ./MACCROBAT2018/18815636.txt  
  inflating: ./MACCROBAT2018/19009665.ann  
  inflating: ./MACCROBAT2018/19009665.t

In [3]:
!pip -q install evaluate accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 32.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [12]:
import os
from typing import List, Dict, Tuple
dataset_folder = '/content/MACCROBAT2018'
file_ids = [f.split(".")[0] for f in os.listdir(dataset_folder) if f.endswith('.txt')]
anno_files = [f+".ann" for f in file_ids]

tags: List[Dict[str, str]] = []
for i in range(len(anno_files)):
    file_path = os.path.join(dataset_folder, anno_files[i])
    with open(file_path, 'r') as f:
        # Work with tags T(T1, T2, ...) only
        text_bound_ann = [t.split("\t") for t in f.read().split("\n") if t.startswith("T")]
        # [['T1', 'Age 2 13', '60-year-old'], ['T2', 'Personal_background 14 23', 'Caucasian'], ....]
        text_bound_lst = []
        for text_b in text_bound_ann:
            label = text_b[1].split(" ")
            try:
                _ = int(label[1])
                _ = int(label[2])
                tag = {
                    "text": text_b[-1],
                    "label": label[0],
                    "start": label[1],
                    "end": label[2]
                }
                # {'text': '60-year-old', 'label': 'Age', 'start': '2', 'end': '13'}
                text_bound_lst.append(tag)
            except:
                pass


{'text': '60-year-old', 'label': 'Age', 'start': '2', 'end': '13'}


In [14]:
import os
from typing import List, Dict, Tuple

class Preprocessing_Maccrobat:
    def __init__(self, dataset_folder, tokenizer):
        self.file_ids = [f.split(".")[0] for f in os.listdir(dataset_folder) if f.endswith('.txt')]

        self.text_files = [f+".txt" for f in self.file_ids]
        self.anno_files = [f+".ann" for f in self.file_ids]

        self.num_samples = len(self.file_ids)

        self.texts: List[str] = []
        # ['Sony', 'International Business Machine Coporation', ...]
        for i in range(self.num_samples):
            file_path = os.path.join(dataset_folder, self.text_files[i])
            with open(file_path, 'r') as f:
                self.texts.append(f.read())

        self.tags: List[Dict[str, str]] = []
        for i in range(self.num_samples):
            file_path = os.path.join(dataset_folder, self.anno_files[i])
            with open(file_path, 'r') as f:
                # Work with tags T(T1, T2, ...) only
                text_bound_ann = [t.split("\t") for t in f.read().split("\n") if t.startswith("T")]
                text_bound_lst = []
                for text_b in text_bound_ann:
                    label = text_b[1].split(" ")
                    try:
                        _ = int(label[1])
                        _ = int(label[2])
                        tag = {
                            "text": text_b[-1],
                            "label": label[0],
                            "start": label[1],
                            "end": label[2]
                        }
                        text_bound_lst.append(tag)
                    except:
                        pass

                self.tags.append(text_bound_lst)
        self.tokenizer = tokenizer

    def process(self) -> Tuple[List[List[str]], List[List[str]]]:
        input_texts = []
        input_labels = []

        for idx in range(self.num_samples):
            full_text = self.texts[idx]
            tags = self.tags[idx]

            label_offset = []
            continuous_label_offset = []
            for tag in tags:
                offset = list(range(int(tag["start"]), int(tag["end"])+1))
                label_offset.append(offset)
                continuous_label_offset.extend(offset)

            all_offset = list(range(len(full_text)))
            zero_offset = [offset for offset in all_offset if offset not in continuous_label_offset]
            zero_offset = Preprocessing_Maccrobat.find_continuous_ranges(zero_offset)

            self.tokens = []
            self.labels = []
            self._merge_offset(full_text, tags, zero_offset, label_offset)
            assert len(self.tokens) == len(self.labels), f"Length of tokens and labels are not equal"

            input_texts.append(self.tokens)
            input_labels.append(self.labels)

        return input_texts, input_labels

    def _merge_offset(self, full_text, tags, zero_offset, label_offset):
        # câu là: work IBM at (work -> 0123); IBM: 4,5,6; at [7, 8] ,....
        # zero_offset = [[0, 1, 2, 3], [7, 8]]
        # add_offset = [[4, 5, 6]]
        i = j = 0
        while i < len(zero_offset) and j < len(label_offset):
            if zero_offset[i][0] < label_offset[j][0]:
                self._add_zero(full_text, zero_offset, i) # thêm vào dưới dạng ko có thực thể
                i += 1
            else:
                self._add_label(full_text, label_offset, j, tags)  # thêm vào dưới dạng có thực thể
                j += 1

        while i < len(zero_offset): # thêm vào dưới dạng ko có thực thể
            self._add_zero(full_text, zero_offset, i)
            i += 1

        while j < len(label_offset): # thêm vào dưới dạng có thực thể
            self._add_label(full_text, label_offset, j, tags)
            j += 1

    def _add_zero(self, full_text, offset, index):
        start, *_ ,end =  offset[index] if len(offset[index]) > 1 else (offset[index][0], offset[index][0]+1)
        text = full_text[start:end]
        text_tokens = self.tokenizer.tokenize(text)

        self.tokens.extend(text_tokens)
        self.labels.extend(
            ["O"]*len(text_tokens)
        )

    def _add_label(self, full_text, offset, index, tags):
        start, *_ ,end =  offset[index] if len(offset[index]) > 1 else (offset[index][0], offset[index][0]+1)
        text = full_text[start:end]
        text_tokens = self.tokenizer.tokenize(text)

        self.tokens.extend(text_tokens)
        self.labels.extend(
            [f"B-{tags[index]['label']}"] + [f"I-{tags[index]['label']}"]*(len(text_tokens)-1)
        )

    @staticmethod
    def build_label2id(tokens: List[List[str]]):
        label2id = {}
        id_counter = 0
        for token in [token for sublist in tokens for token in sublist]:
            if token not in label2id:
                label2id[token] = id_counter
                id_counter += 1
        return label2id

    @staticmethod
    def find_continuous_ranges(data: List[int]):
        if not data:
            return []
        ranges = []
        start = data[0]
        prev = data[0]
        for number in data[1:]:
            if number != prev + 1:
                ranges.append(list(range(start, prev + 1)))
                start = number
            prev = number
        ranges.append(list(range(start, prev + 1)))
        return ranges



In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("d4data/biomedical-ner-all")

dataset_folder = "./MACCROBAT2018"

Maccrobat_builder = Preprocessing_Maccrobat(dataset_folder, tokenizer)
input_texts, input_labels = Maccrobat_builder.process()

label2id = Preprocessing_Maccrobat.build_label2id(input_labels)
id2label = {v: k for k, v in label2id.items()}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

## 2. Dataset loader

In [16]:
from sklearn.model_selection import train_test_split


inputs_train, inputs_val, labels_train, labels_val = train_test_split(
    input_texts,
    input_labels,
    test_size=0.2,
    random_state=42
)

In [17]:
import torch
from torch.utils.data import Dataset

MAX_LEN = 512

class NER_Dataset(Dataset):
    def __init__(self, input_texts, input_labels, tokenizer, label2id, max_len=MAX_LEN):
        super().__init__()
        self.tokens = input_texts
        self.labels = input_labels
        self.tokenizer = tokenizer
        self.label2id = label2id
        self.max_len = max_len

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        input_token = self.tokens[idx]
        label_token = [self.label2id[label] for label in self.labels[idx]]

        input_token = self.tokenizer.convert_tokens_to_ids(input_token)
        attention_mask = [1] * len(input_token)

        input_ids = self.pad_and_truncate(input_token, pad_id= self.tokenizer.pad_token_id)
        labels = self.pad_and_truncate(label_token, pad_id=0)
        attention_mask =  self.pad_and_truncate(attention_mask, pad_id=0)

        return {
            "input_ids": torch.as_tensor(input_ids),
            "labels": torch.as_tensor(labels),
            "attention_mask": torch.as_tensor(attention_mask)
            }

    def pad_and_truncate(self, inputs: List[int], pad_id: int):
        if len(inputs) < self.max_len:
            padded_inputs = inputs + [pad_id] * (self.max_len - len(inputs))
        else:
            padded_inputs = inputs[:self.max_len]
        return padded_inputs

    def label2id(self, labels: List[str]):
        return [self.label2id[label] for label in labels]

In [18]:
train_set = NER_Dataset(inputs_train, labels_train, tokenizer, label2id)
val_set = NER_Dataset(inputs_val, labels_val, tokenizer, label2id)

## 3. Model

In [19]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained(
    "d4data/biomedical-ner-all",
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True
)
model

config.json:   0%|          | 0.00/5.00k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/266M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at d4data/biomedical-ner-all and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([84]) in the checkpoint and torch.Size([83]) in the model instantiated
- classifier.weight: found shape torch.Size([84, 768]) in the checkpoint and torch.Size([83, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
    

## 4. Training

In [20]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    mask = labels != 0
    predictions = np.argmax(predictions, axis=-1)
    return accuracy.compute(predictions=predictions[mask], references=labels[mask])

In [28]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="out_dir",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    optim="adamw_torch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    tokenizer = tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.601869,0.760409
2,No log,0.567805,0.784480
3,No log,0.601000,0.767751
4,No log,0.599606,0.780390
5,No log,0.642134,0.783086
6,No log,0.637387,0.784851
7,No log,0.627792,0.780112
8,No log,0.638033,0.784851
9,No log,0.651468,0.784201
10,No log,0.669824,0.783550


TrainOutput(global_step=200, training_loss=0.0862648105621338, metrics={'train_runtime': 232.6661, 'train_samples_per_second': 13.754, 'train_steps_per_second': 0.86, 'total_flos': 418702245888000.0, 'train_loss': 0.0862648105621338, 'epoch': 20.0})

## 5. Inference

In [29]:
test_sentence = """A 48 year - old female presented with vaginal bleeding and abnormal Pap smears .
Upon diagnosis of invasive non - keratinizing SCC of the cervix ,
she underwent a radical hysterectomy with salpingo - oophorectomy
which demonstrated positive spread to the pelvic lymph nodes and the parametrium .
Pathological examination revealed that the tumour also extensively involved the lower uterine segment .
"""
input = torch.as_tensor([tokenizer.convert_tokens_to_ids(test_sentence.split())])

In [30]:
input = input.to("cuda")

In [31]:
outputs = model(input)
outputs.logits.shape

torch.Size([1, 63, 83])

In [32]:
_, preds = torch.max(outputs.logits, -1)
preds = preds[0].cpu().numpy()

In [33]:
for token, pred in zip(test_sentence.split(), preds):
    print(f"{token}\t{id2label[pred]}")

A	O
48	B-Age
year	I-Age
-	I-Age
old	I-Age
female	B-Sex
presented	B-Clinical_event
with	O
vaginal	O
bleeding	B-Sign_symptom
and	O
abnormal	B-Sign_symptom
Pap	O
smears	O
.	O
Upon	O
diagnosis	O
of	O
invasive	B-Detailed_description
non	O
-	I-Detailed_description
keratinizing	I-Detailed_description
SCC	I-Detailed_description
of	O
the	O
cervix	O
,	O
she	O
underwent	O
a	O
radical	B-Detailed_description
hysterectomy	I-Detailed_description
with	O
salpingo	B-Detailed_description
-	I-Detailed_description
oophorectomy	I-Detailed_description
which	O
demonstrated	O
positive	B-Lab_value
spread	O
to	O
the	O
pelvic	B-Biological_structure
lymph	I-Biological_structure
nodes	O
and	O
the	O
parametrium	O
.	O
Pathological	B-Diagnostic_procedure
examination	B-Diagnostic_procedure
revealed	O
that	O
the	O
tumour	B-Coreference
also	O
extensively	O
involved	O
the	O
lower	O
uterine	I-Biological_structure
segment	O
.	O


In [34]:
for token, pred in zip(tokenizer.tokenize(test_sentence), preds):
    print(f"{token}\t{id2label[pred]}")

a	O
48	B-Age
year	I-Age
-	I-Age
old	I-Age
female	B-Sex
presented	B-Clinical_event
with	O
va	O
##ginal	B-Sign_symptom
bleeding	O
and	B-Sign_symptom
abnormal	O
pa	O
##p	O
sm	O
##ears	O
.	O
upon	B-Detailed_description
diagnosis	O
of	I-Detailed_description
invasive	I-Detailed_description
non	I-Detailed_description
-	O
ke	O
##rat	O
##ini	O
##zing	O
sc	O
##c	O
of	B-Detailed_description
the	I-Detailed_description
ce	O
##r	B-Detailed_description
##vi	I-Detailed_description
##x	I-Detailed_description
,	O
she	O
underwent	B-Lab_value
a	O
radical	O
h	O
##yst	B-Biological_structure
##ere	I-Biological_structure
##ct	O
##omy	O
with	O
sal	O
##ping	O
##o	B-Diagnostic_procedure
-	B-Diagnostic_procedure
o	O
##op	O
##hore	O
##ct	B-Coreference
##omy	O
which	O
demonstrated	O
positive	O
spread	O
to	I-Biological_structure
the	O
pe	O
